<img src="img/logo.svg" width="400" align="right">

# ICOS Carbon Portal Python Library
# Example: Explore ICOS Stations


## Documentation
Full documentation for the library on the[project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on[pypi.org](https://pypi.org/project/icoscp/"), source available on[github](https://github.com/ICOS-Carbon-Portal/pylib)

## Import the library

In [2]:
from icoscp.station import station

## List of stations
Get a list of all ICOS stations as a pandas dataframe table with basic information.

In [ ]:
station.getIdList()

## Map of stations
Generate a folium map of stations. Each marker in the map represents a station and contains station related information.
The `project` argument could be any one of the following:  
`'ALL', 'ICOS', 'NEON', 'INGOS', 'FLUXNET'`.

In [ ]:
station.getIdList(project='ALL', outfmt='map')

## Station meta data
Extract a station and display corresponding info, data products and a list of data objects

In [ ]:
myStation = station.get('ZEP')
myStation.info()

In [ ]:
myStation.products()

In [ ]:
myStation.data(2).tail()

## Station data object identifiers
Extract all level 2 data object identifiers. This is the id (PID) you need to extract the actual values (data), as shown in the example 1: `ex1_data`

In [ ]:
myStation.data(2)['dobj'].values